In [ ]:
%%capture
from remote_read_sql import get_db_connection
from pathlib import Path
import pandas as pd
from clinicedc_constants import MICROMOLES_PER_LITER
from clinicedc_utils import EgfrCockcroftGault
from edc_pdutils.utils import convert_visit_code_to_float
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from scipy import stats
pd.set_option("future.no_silent_downcasting", True)

In [ ]:
# change as needed
my_cnf_path = Path("~/.my.cnf")
my_cnf_connection_name = "client"
data_folder = Path("~/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Documents - igh.respond-africa/META2-data/").expanduser()

In [ ]:
db_conn_opts = dict(my_cnf_path=my_cnf_path, connection_name=my_cnf_connection_name, db_name="meta2_production", local_bind_port=3306)

In [ ]:
# sql statements
sql_rft = "select subject_visit_id, creatinine_value, creatinine_units, creatinine_abnormal, creatinine_reportable, creatinine_grade, egfr_value, egfr_units, egfr_abnormal, egfr_reportable, egfr_grade from meta_subject_bloodresultsrft"
sql_visit = "select id, subject_identifier, visit_code, visit_code_sequence, report_datetime, site_id from meta_subject_subjectvisit"
sql_consent = "select subject_identifier, gender, dob from meta_consent_subjectconsent"
sql_physical_exam = "select subject_visit_id, weight from meta_subject_physicalexam"
sql_followupvitals = "select subject_visit_id, weight from meta_subject_followupvitals"
sql_screen = "select screening_identifier, subject_identifier, weight as weight_scr, selection_method from meta_screening_subjectscreening"
sql_dd = "select model, field_name, field_type, `default`, prompt from edc_data_manager_datadictionary"

In [ ]:
# read in table data
with get_db_connection(**db_conn_opts) as db_conn:
    df_rft = pd.read_sql(sql_rft, con=db_conn)
    df_visit = pd.read_sql(sql_visit, con=db_conn)
    df_consent = pd.read_sql(sql_consent, con=db_conn)
    df_physical_exam = pd.read_sql(sql_physical_exam, con=db_conn)
    df_followupvitals = pd.read_sql(sql_followupvitals, con=db_conn)
    df_screen = pd.read_sql(sql_screen, con=db_conn)


In [ ]:
df_screen["selection_method"] = df_screen["selection_method"].replace('purposively_selected', 'purposeful')
df_screen["selection_method"] = df_screen["selection_method"].replace('random_sampling', 'random')
cat_type = pd.CategoricalDtype(categories=["purposeful", "random"], ordered=True)
df_screen["selection_method"] = df_screen["selection_method"].astype(cat_type)
df_screen.selection_method.value_counts()


In [ ]:
# read rando_meta2.csv
df_rando = pd.read_csv(data_folder / "rando" / "rando_meta2.csv",sep="|", encoding="utf8")
df_rando["assignment"] = df_rando["assignment"].replace('placebo', 'control')
df_rando["assignment"] = df_rando["assignment"].replace('active', 'treatment')
assignment_type = pd.CategoricalDtype(categories=["control", "treatment"], ordered=True)
df_rando["assignment"] = df_rando["assignment"].astype(assignment_type)
df_rando

In [ ]:
# convert visit code to a float
df_visit = df_visit.rename(columns={"id": "subject_visit_id", "report_datetime": "visit_datetime"})
df_visit["visit_code_str"] = df_visit["visit_code"]
convert_visit_code_to_float(df_visit)

# convert consent DOB to a datetime
df_consent["dob"] = df_consent['dob'].apply(pd.to_datetime, errors="coerce")

In [ ]:
# merge to get all longitudunal weight values per subject
df_weight = pd.merge(df_physical_exam, df_followupvitals, on=["subject_visit_id"], how="outer")
df_weight = df_visit[["subject_visit_id", "subject_identifier", "visit_datetime"]].merge(df_weight, on="subject_visit_id", how="left")
df_weight["weight"] = df_weight['weight_x'].combine_first(df_weight['weight_y'])

df_weight["weight_imputed"] = df_weight.groupby(by=["subject_identifier"])['weight'].ffill()
df_last = df_weight[df_weight["weight_imputed"].notna()].groupby('subject_identifier').last(numeric_only=True).reset_index()
df_first = df_weight[df_weight["weight_imputed"].notna()].groupby('subject_identifier').first(numeric_only=True).reset_index()
df_b2d = pd.merge(df_first[["subject_identifier", "weight_imputed"]], df_last[["subject_identifier", "weight_imputed"]], on='subject_identifier', how="outer", suffixes=("_baseline", "_endline"))
df_b2d = df_b2d.rename(columns={"weight_imputed_baseline": "weight_baseline_imputed", "weight_imputed_endline":"weight_endline_imputed"})
df_weight = df_weight.merge(df_b2d, on="subject_identifier", how="left").sort_values(by=["subject_identifier"]).reset_index(drop=True)
df_weight["weight_change_b2e_imputed"] = df_weight["weight_endline_imputed"] - df_weight["weight_baseline_imputed"]
df_weight = df_weight.sort_values(by=["subject_identifier", "visit_datetime"]).reset_index(drop=True)
df_weight['weight_change'] = df_weight.groupby('subject_identifier')['weight_imputed'].diff()
df_weight['weight_change'] = df_weight.weight_change.apply(lambda x: 0.0 if pd.isna(x) else x)
df_weight = df_weight.drop(columns=["subject_identifier", "visit_datetime", "weight_x", "weight_y"])



In [ ]:
df_weight

In [ ]:
# merge tables
df = (df_visit
    .merge(df_screen, on="subject_identifier", how="left")
    .merge(df_consent, on="subject_identifier", how="left")
    .merge(df_rft, on="subject_visit_id", how="left")
    .merge(df_weight, on="subject_visit_id", how="left")
    .sort_values(by=["subject_identifier", "visit_datetime"])
    .reset_index(drop=True)
)

In [ ]:
df_b2d = df.groupby('subject_identifier')['visit_datetime'].agg(visit_datetime_baseline=('min'), visit_datetime_endline=('max'))
df = (df
    .merge(df_b2d, on="subject_identifier", how="left")
    .merge(df_rando[["subject_identifier", "assignment", "allocation", "allocated_datetime"]], on="subject_identifier", how="left")
    .sort_values(by=["subject_identifier"]).reset_index(drop=True)
    .reset_index(drop=True)
      )
df["days_on_study"] = (df["visit_datetime_endline"] - df["visit_datetime_baseline"]).dt.days
df["days_since_baseline"] = (df["visit_datetime"] - df["visit_datetime_baseline"]).dt.days


In [ ]:
# calc age in years
df["age_in_years"] = (df['visit_datetime'] - df['dob'])/pd.Timedelta(days=365.25)

In [ ]:
df["days_since_baseline"].describe()


In [ ]:
df

In [ ]:
df_model = (
    df[["subject_identifier", "weight_imputed", "days_since_baseline", "assignment", "gender", "age_in_years"]]
    .copy()
    .dropna()
    .sort_values(["subject_identifier", "days_since_baseline"])
    .reset_index(drop=True)
)

df_model["subject_identifier"] = df_model["subject_identifier"].astype(str)
df_model.dtypes


In [ ]:
model = smf.mixedlm(
    # formula='weight_imputed ~ days_since_baseline + C(assignment) + C(gender) + age_in_years',
    # formula='weight_imputed ~ days_since_baseline + C(assignment) + C(gender) + I(age_in_years**2)',
    formula='weight_imputed ~ days_since_baseline * C(gender) + C(assignment)',
    data=df_model,
    groups=df_model['subject_identifier'],
)

In [ ]:
result = model.fit()

In [ ]:
result.summary().tables[1]

In [ ]:
df_baseline = df[df['days_since_baseline'] == 0].copy()

In [ ]:
weight_purposeful = df_baseline[df_baseline['selection_method'] == 'purposeful']['weight_imputed']
weight_random = df_baseline[df_baseline['selection_method'] == 'random']['weight_imputed']
ttest_weight = stats.ttest_ind(weight_purposeful, weight_random, equal_var=False)

In [ ]:
print(f"Mean Purposeful: {weight_purposeful.mean():.2f}")
print(f"Mean Random:     {weight_random.mean():.2f}")
print(f"T-test P-value:  {ttest_weight.pvalue:.4f}")

In [ ]:
purposeful = df_baseline[df_baseline['selection_method'] == 'purposeful']['age_in_years']
random = df_baseline[df_baseline['selection_method'] == 'random']['age_in_years']
ttest_result = stats.ttest_ind(purposeful, random, equal_var=False)

In [ ]:
print("age_in_years")
print(f"Mean Purposeful: {purposeful.mean():.2f}")
print(f"Mean Random:     {random.mean():.2f}")
print(f"T-test P-value:  {ttest_result.pvalue:.4f}")

In [ ]:
df_screen.select_dtypes(["int64", "float64"])

In [ ]:

cols = [
"weight_scr",
"creatinine_value",
"egfr_value",
"weight",
"weight_baseline_imputed",
"days_on_study",
"age_in_years",
]
result_text = []
for col in cols:
    purposeful = df_baseline[df_baseline['selection_method'] == 'purposeful'][col]
    random = df_baseline[df_baseline['selection_method'] == 'random'][col]
    ttest_result = stats.ttest_ind(purposeful, random, equal_var=False)
    result_text.append(
        f"{col}\n"
        f"purposeful: {len(purposeful)}\n"
        f"random: {len(random)}\n"
        f"Mean Purposeful: {purposeful.mean():.2f}\n"
        f"Mean Random:     {random.mean():.2f}\n"
        f"T-test P-value:  {ttest_result.pvalue:.4f}\n"
    )
print("\n\n".join(result_text))

In [ ]:
print("gender")
print(f"Mean Purposeful: {purposeful.mean():.2f}")
print(f"Mean Random:     {random.mean():.2f}")
print(f"T-test P-value:  {ttest_result.pvalue:.4f}")

In [ ]:
# recalculate eGFR
def calculate_egfr(s):
    if not s["creatinine_value"]:
        return pd.NA
    else:
        obj = EgfrCockcroftGault(
            gender=s["gender"],
            age_in_years=s["age_in_years"],
            weight=s["weight"],
            creatinine_value=s["creatinine_value"],
            creatinine_units=MICROMOLES_PER_LITER,
        )
    return obj.value

df["egfr_value_recalc"] = pd.to_numeric(df.apply(lambda r: calculate_egfr(r), axis=1))

In [ ]:
# create categorical column `egfr_stage` using KIDGO
bins = [0, 15, 30, 45, 60, 90, 200]
labels = ['G5', 'G4', 'G3b', 'G3a', 'G2', 'G1']
df['egfr_stage'] = pd.cut(
    df['egfr_value_recalc'],
    bins=bins,
    labels=labels,
    right=False, # [a, b) -> G4 is [15, 30) -> 15.0 is G4, but 30.0 is G3b.
    include_lowest=True # (0) is included in the first bin (G5)
)
kdigo_order = ['G1', 'G2', 'G3a', 'G3b', 'G4', 'G5']
df['egfr_stage'] = pd.Categorical(
    df['egfr_stage'],
    categories=kdigo_order,
    ordered=True
    )


In [ ]:
df["creatinine_grade"] = df.creatinine_grade.apply(lambda x: "" if pd.isna(x) else x )
df["egfr_abnormal"] = df.egfr_abnormal.apply(lambda x: "" if pd.isna(x) else x )
df["egfr_reportable"] = df.egfr_reportable.apply(lambda x: "" if pd.isna(x) else x )
df["egfr_grade"] = df.egfr_grade.apply(lambda x: "" if pd.isna(x) else x )

# recalc grade

In [ ]:
# export as CSV
df.to_csv(data_folder / "egfr_202510", "egfr.csv", sep="|", index=False, encoding="utf8")

# export as STATA
df.to_stata(data_folder / "egfr_202510", "egfr.dta", version=118, write_index=False)

In [ ]:
def plot_numericals(df, *numerical_cols):
    # Determine plot size (optional, but good for multiple plots)
    num_plots = len(numerical_cols)
    cols = 3  # Columns per figure
    rows = (num_plots + cols - 1) // cols

    # Create a figure to hold all subplots
    fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 4 * rows))
    axes = axes.flatten() # Flatten the 2D array of axes for easy indexing

    for i, col in enumerate(numerical_cols):
        ax = axes[i]

        sns.histplot(
                data=df,
                x=col,
                kde=True,  # This tells Seaborn to draw the smoothing line
                ax=ax,
                bins=20,
                stat='density' # Normalize the histogram height to match the KDE scale
            )

        # Plot KDE (Kernel Density Estimate) - requires calculation or a library like seaborn
        # For simplicity with matplotlib only, we'll focus on the histogram.
        # If using seaborn: sns.histplot(df[col], kde=True, ax=ax)

        upper_limit = df[col].quantile(0.99)
        plt.xlim(
            df[col].min(),  # Start at the minimum value
            upper_limit     # End at the 99th percentile
        )
        ax.set_title(f'Distribution of {col}')
        ax.set_xlabel(col)
        ax.set_ylabel('Density')
        # ax.grid(axis='y', alpha=0.3)
        ax.grid(axis='y', alpha=0.5)

    # Hide any unused subplots
    for j in range(num_plots, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.savefig('all_numeric_distributions.png')
    plt.show()

In [ ]:
def plot_categorical_distribution(df, category_column, category_order,  title='Distribution of Categories'):
    """Generates a bar plot for the specified categorical column."""

    plot_order = category_order
    x_label = 'eGFR KDIGO Stage'

    plt.figure(figsize=(6,4))

    # 2. Use seaborn.countplot for easy counting and plotting
    ax = sns.countplot(
        data=df,
        x=category_column,
        order=plot_order, # Use the defined or derived order for the x-axis
        # palette='viridis',
        # hue=category_column,
        # legend=False,
    )

    # 3. Add counts and percentages on top of the bars for clarity
    total = len(df)
    for p in ax.patches:
        height = p.get_height()
        if pd.notna(height): # Check for NaN if the category was empty
            ax.annotate(
                f'{int(height)}\n({height/total:.1%})', # Show count and percentage
                (p.get_x() + p.get_width() / 2., height),
                ha='center',
                va='bottom',
                xytext=(0, 5),
                textcoords='offset points'
            )

    # 4. Final plot customizations
    # ax.set_title(title, fontsize=16)
    ax.set_xlabel(x_label, fontsize=12)
    ax.set_ylabel('Subjects (count)', fontsize=12)

    plt.grid(axis='y', linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.show()


In [ ]:
plot_numericals(df, 'creatinine_value', 'weight', 'weight_imputed', 'age_in_years')

In [ ]:
df['egfr_stage'].value_counts()

In [ ]:
plot_categorical_distribution(df.query("egfr_stage in @kdigo_order"), 'egfr_stage', kdigo_order)

In [ ]:
def visualize_weight_change(df, change_column='weight_change'):
    """Plots the distribution of weight change with a clear zero reference."""

    plt.figure(figsize=(6, 4))

    # Use sns.histplot to plot the distribution with a KDE curve
    ax = sns.histplot(
        data=df,
        x=change_column,
        kde=True, # Overlay a smooth distribution line
        bins=20,  # Adjust bins as needed for your data granularity
        stat='count' # Show raw counts on the y-axis
    )

    # Add a vertical line at x=0 to clearly separate gain and loss
    ax.axvline(0, color='red', linestyle='--', linewidth=2, label='No Change (0)')

    # Add shading to distinguish loss and gain areas (Optional, but very clear)
    # Get the minimum and maximum change values for shading range
    min_val = df[change_column].min()
    max_val = df[change_column].max()

    # Shade the Loss Area (left of 0)
    ax.axvspan(min_val, 0, color='salmon', alpha=0.15, label='Weight Loss')

    # Shade the Gain Area (right of 0)
    ax.axvspan(0, max_val, color='skyblue', alpha=0.15, label='Weight Gain')

    # Customize labels and title
    # ax.set_title('Weight Change (Final - Initial)', fontsize=16)
    ax.set_xlabel('Weight Change (kg )', fontsize=12)
    ax.set_ylabel('Subjects (Count)', fontsize=12)

    ax.legend(loc='upper right')
    plt.tight_layout()
    plt.savefig('weight_change_distribution.png')

    # Print summary statistics for context
    num_gain = len(df[df[change_column] > 0])
    num_loss = len(df[df[change_column] < 0])
    num_total = len(df)

    print(f"Total Subjects: {num_total}")
    print(f"Subjects Gaining Weight: {num_gain} ({num_gain/num_total:.1%})")
    print(f"Subjects Losing Weight: {num_loss} ({num_loss/num_total:.1%})")
    print(f"Mean Change: {df[change_column].mean():.2f}")


# Example of how you would call this function:
# visualize_weight_change(df, change_column='weight_change')

In [ ]:
visualize_weight_change(df, change_column='weight_change')

In [ ]:
visualize_weight_change(df.query("egfr_stage in @kdigo_order"), change_column='weight_change')
